In [1]:
%reload_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, '/home/yuncong/project/cython-munkres-wrapper/build/lib.linux-x86_64-2.7')
from munkres import munkres

import os
import time

from utilities2015 import *

from scipy.spatial.distance import cdist, pdist, squareform
from scipy.cluster.hierarchy import average, fcluster, single, complete

from joblib import Parallel, delayed

from skimage.color import gray2rgb
from skimage.measure import find_contours
from skimage.util import img_as_float

import matplotlib.pyplot as plt
%matplotlib inline

from networkx import from_dict_of_lists, Graph, adjacency_matrix, dfs_postorder_nodes
from networkx.algorithms import node_connected_component

In [2]:
from shape_matching import *

In [33]:
stack_name = 'MD593'

dms = dict([(sec_ind, DataManager(stack=stack_name, section=sec_ind, segm_params_id='tSLIC200')) 
       for sec_ind in [138, 139,140, 141, 142]])

In [15]:
def boundary_distance(b1, b2, sc1=None, sc2=None, loc_thresh=1500, verbose=False):
    '''
    Compute the distance between two boundaries.
    Each tuple consists of (edgeSet, interior_texture, exterior_textures, points, center)
    
    Parameters
    ----------
    b1 : tuple
    b2 : tuple
    sc1 : #points-by-32 array
        pre-computed shape context descriptor
    sc2 : #points-by-32 array
        pre-computed shape context descriptor
    '''
    
    _, interior_texture1, exterior_textures1, points1, center1, \
                        majorv1, minorv1, majorlen1, minorlen1, ell_center1 = b1
        
    _, interior_texture2, exterior_textures2, points2, center2, \
                        majorv2, minorv2, majorlen2, minorlen2, ell_center2 = b2
    
    if sc1 is not None:
        assert len(sc1) == points1.shape[0], 'number mismatch %d %d'%(len(sc1), points1.shape[0])
    
    if sc2 is not None:
        assert len(sc2) == points2.shape[0], 'number mismatch %d %d'%(len(sc2), points2.shape[0])

    # compute location difference
    d_loc = np.linalg.norm(center1 - center2)
    D_loc = np.maximum(0, d_loc - 500)
    
#     sys.stderr.write('D_loc computed\n')
#     print 'd_loc', d_loc

    if d_loc > loc_thresh:
        return np.inf, np.inf, np.inf, np.inf, np.inf
    
    n1 = len(points1)
    n2 = len(points2)
    
    if max(n1,n2) > min(n1,n2) * 3:
        return np.inf, np.inf, np.inf, np.inf, np.inf
    
    # compute interior texture difference
    D_int = chi2(interior_texture1, interior_texture2)
#     D_ext = hausdorff_histograms(exterior_textures1, exterior_textures2, metric=chi2)

#     sys.stderr.write('D_interior computed\n')

    # compute shape difference, exterior texture difference
    D_shape, matches = shape_context_score(points1, points2, descriptor1=sc1, descriptor2=sc2, verbose=verbose)
#         D_ext = np.mean([chi2(exterior_textures1[i], exterior_textures2[j]) for i, j in matches])
    
#     sys.stderr.write('D_shape computed\n')
    
    bg_match = 0

    if len(matches) == 0:
        D_ext = np.inf
    else:
        ddd = []
        for i, j in matches:
            # -1 vs -1
            if np.isnan(exterior_textures1[i]).all() and np.isnan(exterior_textures2[j]).all():
                s = 0
                bg_match += 1
                ddd.append(s)
            # non -1 vs non -1
            elif not np.isnan(exterior_textures1[i]).all() and not np.isnan(exterior_textures2[j]).all():
                s = chi2(exterior_textures1[i], exterior_textures2[j])
                if verbose:
                    print 'exterior', i,j,s
                ddd.append(s)
            # -1 vs non -1
            else:
                ddd.append(2.)

        if len(ddd) == 0:
            D_ext = np.inf
        elif len(ddd) == bg_match:
            D_ext = 2.
        else:
            D_ext = np.mean(ddd)
    
    D_shape = D_shape * .01

    # weighted average of four terms
    d = D_int + D_ext + D_shape + 0 * D_loc
    
    return d, D_int, D_ext, D_shape, D_loc


In [16]:
def shape_context_score(pts1, pts2, descriptor1=None, descriptor2=None, verbose=False):

    if descriptor1 is None:
        descriptor1 = compute_shape_context_descriptors(pts1, dist_limit=.8)
    
    if descriptor2 is None:
        descriptor2 = compute_shape_context_descriptors(pts2, dist_limit=.8)
        
    descriptor_dists = cdist(descriptor1, descriptor2, metric='euclidean')
        
#     b = time.time()

    T, best_match, best_sample, best_score = ransac_compute_rigid_transform(descriptor_dists, pts1, pts2, 
                                                                            ransac_iters=50, confidence_thresh=0.03, 
                                                                            sample_size=3, matching_iter=10,
                                                                           n_neighbors=3)
#     print 'ransac_compute_rigid_transform', time.time() - b

    
    if T is None and len(best_match)==0:
        return np.inf, []
    
    if verbose:
        print 'best_match', best_match
        print 'best_sample', best_sample
        print 'best_score', best_score

    return best_score, best_match

In [17]:
from skimage.util import pad

def generate_matching_visualizations(sec1, sec2, matchings=None):
    '''
    Generate visualization for matching between sec1 and sec2
    '''
    
    dm1 = dms[sec1]
    dm2 = dms[sec2]
    
    boundaries1 = dm1.load_pipeline_result('boundaryModels')
    boundaries2 = dm2.load_pipeline_result('boundaryModels')
    
    if matchings is None:
        matchings = dm1.load_pipeline_result('matchings%dWith%d'%(sec1, sec2))

    matched_boundaries1 = [boundaries1[i][0] for ind, (d,i,j) in enumerate(matchings)]
    vis_matched_boundaries_next = dm1.visualize_edge_sets(matched_boundaries1, show_set_index=True)

    matched_boundaries2 = [boundaries2[j][0] for ind, (d,i,j) in enumerate(matchings)]
    vis_matched_boundaries_prev = dm2.visualize_edge_sets(matched_boundaries2, show_set_index=True)

    # Place two images vertically 
    h1, w1 = vis_matched_boundaries_next.shape[:2]
    h2, w2 = vis_matched_boundaries_prev.shape[:2]
    
    if w1 < w2:
        left_margin = int((w2 - w1)/2)
        right_margin = w2 - w1 - left_margin
        vis_matched_boundaries_next = pad(vis_matched_boundaries_next, 
                                          ((0,0),(left_margin,right_margin),(0,0)), 
                                          'constant', constant_values=255)
    else:
        left_margin = int((w1 - w2)/2)
        right_margin = w1 - w2 - left_margin
        vis_matched_boundaries_prev = pad(vis_matched_boundaries_prev, 
                                          ((0,0),(left_margin,right_margin),(0,0)), 
                                          'constant', constant_values=255)
        
    vis = np.r_[vis_matched_boundaries_next, vis_matched_boundaries_prev]
    
    return vis
    
#     dm1.save_pipeline_result(vis, 'matchedBoundaries%dWith%dSideBySide'%(sec1, sec2), 'jpg')
#     dm2.save_pipeline_result(vis, 'matchedBoundaries%dWith%dSideBySide'%(sec2, sec1), 'jpg')
    
#     for ind, (s, i, j) in enumerate(matchings):
#         # image with a particular boundary highlighted
        
#         vis1 = dm1.load_pipeline_result('boundary%02d'%i, 'jpg')
#         vis2 = dm2.load_pipeline_result('boundary%02d'%j, 'jpg')
        
#         h1, w1 = vis1.shape[:2]
#         h2, w2 = vis2.shape[:2]
        
#         # Place vertically
#         if w1 < w2:
#             left_margin = int((w2 - w1)/2)
#             right_margin = w2 - w1 - left_margin
#             vis1 = pad(vis1, ((0,0),(left_margin,right_margin),(0,0)), 'constant', constant_values=255)
#         else:
#             left_margin = int((w1 - w2)/2)
#             right_margin = w1 - w2 - left_margin
#             vis2 = pad(vis2, ((0,0),(left_margin,right_margin),(0,0)), 'constant', constant_values=255)

#         vis_pair_prev = np.r_[vis1, vis2]
#         vis_pair_next = np.r_[vis2, vis1]
        
#         dm1.save_pipeline_result(vis_pair_prev, 'matchedBoundaryPair%dFor%dWith%d'%(ind, sec1, sec2), 'jpg')
#         dm2.save_pipeline_result(vis_pair_next, 'matchedBoundaryPair%dFor%dWith%d'%(ind, sec2, sec1), 'jpg')
    

In [ ]:
dm1 = dms[139]
dm2 = dms[140]
    
boundaries1 = dm1.load_pipeline_result('boundaryModels')
boundaries2 = dm2.load_pipeline_result('boundaryModels')

sc1 = dm1.load_pipeline_result('shapeContext')
sc2 = dm2.load_pipeline_result('shapeContext')

In [ ]:
es1, interior_texture1, exterior_textures1, points1, center1 = boundaries1[10]
es2, interior_texture2, exterior_textures2, points2, center2 = boundaries2[4]

descriptor_dists = cdist(sc1[10], sc2[4], metric='euclidean')

In [ ]:
T, best_match, best_sample, best_score = ransac_compute_rigid_transform(descriptor_dists, points1, points2, 
                                                                    ransac_iters=50, confidence_thresh=0.03, 
                                                                    sample_size=3, matching_iter=10,
                                                                   n_neighbors=3, verbose=True)

In [ ]:
res = boundary_distance(boundaries1[1], boundaries2[23], sc1=sc1[1], sc2=sc2[23], verbose=True)

In [ ]:
D_boundaries, D_int, D_ext, D_shape = compute_boundary_distances(139, 140, verbose=True)

In [18]:
def compute_boundary_distances(sec1, sec2, verbose=False):
    
    dm1 = dms[sec1]
    dm2 = dms[sec2]
    
    boundaries1 = dm1.load_pipeline_result('boundaryModels')
    boundaries2 = dm2.load_pipeline_result('boundaryModels')
    
    sc1 = dm1.load_pipeline_result('shapeContext')
    sc2 = dm2.load_pipeline_result('shapeContext')

    n_boundaries1 = len(boundaries1)
    n_boundaries2 = len(boundaries2)
    
#     Ds = Parallel(n_jobs=16)(delayed(boundary_distance)(boundaries1[i], boundaries2[j], sc1=sc1[i], sc2=sc2[j]) 
#                              for i, j in product(range(n_boundaries1), range(n_boundaries2)))

    center_dist_thresh = 1500
    
    centers1 = [b[4] for b in boundaries1]
    centers2 = [b[4] for b in boundaries2]
    center_distances = cdist(centers1, centers2, metric='euclidean')
    b1s, b2s = np.where(center_distances < center_dist_thresh)
    
    b = time.time()

#     Ds = []
#     for i, j in zip(b1s, b2s):
#         sys.stderr.write('%d,%d\n'%(i,j))
#         d = boundary_distance(boundaries1[i], boundaries2[j], sc1=sc1[i], sc2=sc2[j], verbose=verbose)
#         Ds.append(d)

    Ds = Parallel(n_jobs=16)(delayed(boundary_distance)(boundaries1[i], boundaries2[j], 
                                                        sc1=sc1[i], sc2=sc2[j], verbose=verbose) 
                             for i, j in zip(b1s, b2s))
    print  'boundary_distance', time.time() - b
    
    D_boundaries = np.inf * np.ones((n_boundaries1, n_boundaries2))
    D_int = np.inf * np.ones((n_boundaries1, n_boundaries2))
    D_ext = np.inf * np.ones((n_boundaries1, n_boundaries2))
    D_shape = np.inf * np.ones((n_boundaries1, n_boundaries2))
    
    D_boundaries[b1s, b2s] = [d for d, d_int, d_ext, d_shape, d_loc in Ds]
    D_int[b1s, b2s] = [d_int for d, d_int, d_ext, d_shape, d_loc in Ds]
    D_ext[b1s, b2s] = [d_ext for d, d_int, d_ext, d_shape, d_loc in Ds]
    D_shape[b1s, b2s] = [d_shape for d, d_int, d_ext, d_shape, d_loc in Ds]
    
#     Ds = []
#     for i, j in product(range(n_boundaries1), range(n_boundaries2)):
#         sys.stderr.write('%d,%d\n'%(i,j))
#         Ds.append(boundary_distance(boundaries1[i], boundaries2[j], sc1=sc1[i], sc2=sc2[j]))

#     D_boundaries, D_int, D_ext, D_shape, D_loc = zip(*Ds)

#     D_boundaries = np.reshape(D_boundaries, (n_boundaries1, n_boundaries2))
#     D_int = np.reshape(D_int, (n_boundaries1, n_boundaries2))
#     D_ext = np.reshape(D_ext, (n_boundaries1, n_boundaries2))
#     D_shape = np.reshape(D_shape, (n_boundaries1, n_boundaries2))
#     D4 = np.reshape(D_loc, (n_boundaries1, n_boundaries2))
    
#     dm1.save_pipeline_result(D_boundaries, 'DBoundaries%dWith%d'%(sec1, sec2), 'npy')
#     dm1.save_pipeline_result(D_int, 'D1s%dWith%d'%(sec1, sec2), 'npy')
#     dm1.save_pipeline_result(D_ext, 'D2s%dWith%d'%(sec1, sec2), 'npy')
#     dm1.save_pipeline_result(D_shape, 'D3s%dWith%d'%(sec1, sec2), 'npy')

    return D_boundaries, D_int, D_ext, D_shape
    
#     dm2.save_pipeline_result(D_boundaries.T, 'DBoundaries%dWith%d'%(sec2, sec1), 'npy')
#     dm2.save_pipeline_result(D_int.T, 'D1s%dWith%d'%(sec2, sec1), 'npy')
#     dm2.save_pipeline_result(D_ext.T, 'D2s%dWith%d'%(sec2, sec1), 'npy')
#     dm2.save_pipeline_result(D_shape.T, 'D3s%dWith%d'%(sec2, sec1), 'npy')


In [19]:
def compute_shape_context_descriptors_section(sec_ind):
    dm = dms[sec_ind]
    boundaries = dm.load_pipeline_result('boundaryModels', 'pkl')
    descs = [compute_shape_context_descriptors(b[3], dist_limit=.8) for b in boundaries]
    dm.save_pipeline_result(descs, 'shapeContext', 'pkl')
#     return descs

In [20]:
from bipartite_matching import *

In [21]:
def match_landmarks(sec1, sec2, D=None, must_match=[], cannot_match=[]):
    
    dm1 = dms[sec1]
    dm2 = dms[sec2]
    boundaries1 = dm1.load_pipeline_result('boundaryModels')
    boundaries2 = dm2.load_pipeline_result('boundaryModels')
    
    if D is None:
        D = dm1.load_pipeline_result('DBoundaries%dWith%d'%(sec1, sec2))
        
    matchings = knn_matching(D, boundaries1, boundaries2)
    matchings = sorted(matchings)
    
    return matchings

In [ ]:
# first_sec = 60
# last_sec = 150

In [ ]:
# Parallel(n_jobs=16)(delayed(compute_shape_context_descriptors_section)(sec_ind) 
#                     for sec_ind in range(first_sec, last_sec+1));

In [23]:
compute_shape_context_descriptors_section(139)

saved /oasis/projects/nsf/csd395/yuncong/CSHL_data_results/MD593/0139/MD593_0139_lossless_gabor-blueNisslWide-segm-tSLIC200-vq-blueNissl_shapeContext.pkl


In [24]:
compute_shape_context_descriptors_section(140)

saved /oasis/projects/nsf/csd395/yuncong/CSHL_data_results/MD593/0140/MD593_0140_lossless_gabor-blueNisslWide-segm-tSLIC200-vq-blueNissl_shapeContext.pkl


In [34]:
sec1 = 138
sec2 = sec1 + 1

D_boundaries, D_int, D_ext, D_shape = compute_boundary_distances(sec1, sec2, verbose=False)
# matchings = match_landmarks(sec1, sec2, D=D_boundaries)
# viz = generate_matching_visualizations(sec1, sec2, matchings=matchings)

# dms[sec1].save_pipeline_result(D_boundaries, 'DBoundariesNext1')
# dms[sec1].save_pipeline_result(matchings, 'matchingsNext1')
# dms[sec1].save_pipeline_result(viz, 'matchingsVizNext1')

# dms[sec2].save_pipeline_result(D_boundaries.T, 'DBoundariesPrev1')
# matchings = [(d,j,i) for d,i,j in matchings]
# dms[sec2].save_pipeline_result(matchings, 'matchingsPrev1')
# dms[sec2].save_pipeline_result(viz, 'matchingsVizPrev1')

boundary_distance 13.8034598827


In [26]:
print D_boundaries[10].argsort()
print D_int[10].argsort()
print D_ext[10].argsort()
print D_shape[10].argsort()

[61 64 92 98 88 37 22  4 87 81 58 31 60 91 56 29  2 63 59 65 57 66 55 67 68
 69 70 71 62  0 75 73 97 96 95 94 93 90 89 86 85 84 83 82 80 79 78 77 76 54
 74 72 53 49 51 21 20 19 18 17 16 15 14 13 12 11 10  9  8  7  6  5  3  1 23
 24 25 26 50 48 47 46 45 44 43 42 41 52 40 38 36 35 34 33 32 30 28 27 39 99]
[31 61 37 22 92  4 87 64 56 60 88 81 29 98 91 58  2 57 65 62 66 59 55 67 68
 69 70 71 63  0 75 73 97 96 95 94 93 90 89 86 85 84 83 82 80 79 78 77 76 54
 74 72 53 49 51 21 20 19 18 17 16 15 14 13 12 11 10  9  8  7  6  5  3  1 23
 24 25 26 50 48 47 46 45 44 43 42 41 52 40 38 36 35 34 33 32 30 28 27 39 99]
[22 91 37 92 61 31 88 58 64 98 60  4 81 56 87 29  2 63 59 65 57 66 55 67 68
 69 70 71 62  0 75 73 97 96 95 94 93 90 89 86 85 84 83 82 80 79 78 77 76 54
 74 72 53 49 51 21 20 19 18 17 16 15 14 13 12 11 10  9  8  7  6  5  3  1 23
 24 25 26 50 48 47 46 45 44 43 42 41 52 40 38 36 35 34 33 32 30 28 27 39 99]
[61 64 98 92 88 58  4 87 81 37 22 31 60 91 29  2 56 62 63 57 65 66 55 67 68
 69 70 71

In [35]:
matchings = match_landmarks(sec1, sec2, D=D_boundaries)

In [36]:
for m in matchings:
    print m

(0.65436401810237965, 77, 65)
(0.78388283316179863, 7, 19)
(0.81979789039185957, 8, 40)
(0.83044204086958373, 6, 5)
(0.8812383306603061, 83, 84)
(0.88834390070696023, 72, 88)
(0.90436101013334957, 52, 55)
(0.91222865288448163, 58, 46)
(0.97765181735051199, 85, 67)
(0.99275827032103703, 30, 78)
(0.99562377055011764, 20, 72)
(1.0040815266948404, 10, 7)
(1.0192664233273439, 81, 94)
(1.0209595728325684, 45, 27)
(1.0532930088985069, 39, 31)
(1.0611077921349887, 43, 95)
(1.0628524773277059, 5, 28)
(1.0686998513232504, 11, 30)
(1.0785706887947195, 80, 79)
(1.079281841909038, 32, 86)
(1.0858071262180202, 86, 38)
(1.1096731943858908, 15, 13)
(1.1102324257272056, 40, 44)
(1.1127387794904713, 53, 18)
(1.1268341505105171, 59, 70)
(1.1714112556723504, 66, 82)
(1.1805272354687539, 89, 64)
(1.1992584075332813, 87, 59)
(1.2068779081309822, 76, 48)
(1.2242001503085322, 34, 2)
(1.2258534147709397, 64, 10)
(1.2688334259912999, 54, 57)
(1.2983499695441032, 35, 66)
(1.3156984784702332, 31, 50)
(1.327339140

In [37]:
viz = generate_matching_visualizations(sec1, sec2, matchings=matchings[:30])
display(viz)

/oasis/projects/nsf/csd395/yuncong/Brain/notebooks/tmp.jpg

In [ ]:
fig, axes = plt.subplots(2,2,figsize=(10,10));

axes[0,0].matshow(D_boundaries);
axes[0,0].set_title('overall');
# axes[0,0].axis('off');
axes[0,1].matshow(D_int);
axes[0,1].set_title('interior');
# axes[0,1].axis('off');
axes[1,0].matshow(D_ext);
axes[1,0].set_title('exterior');
# axes[1,0].axis('off');
axes[1,1].matshow(D_shape);
axes[1,1].set_title('shape');
# axes[1,1].axis('off');

plt.show();

In [ ]:
def f(sec1):
    if sec1 < last_sec:
        sec2 = sec1 + 1
        D_boundaries, D_int, D_ext, D_shape = compute_boundary_distances(sec1, sec2, verbose=False)
        matchings = match_landmarks(sec1, sec2, D=D_boundaries)
        viz = generate_matching_visualizations(sec1, sec2, matchings=matchings)
        dms[sec1].save_pipeline_result(D_boundaries, 'DBoundariesNext1')
        dms[sec1].save_pipeline_result(matchings, 'matchingsNext1')
        dms[sec1].save_pipeline_result(viz, 'matchingsVizNext1')
        dms[sec2].save_pipeline_result(D_boundaries, 'DBoundariesPrev1')
        dms[sec2].save_pipeline_result(matchings, 'matchingsPrev1')
        dms[sec2].save_pipeline_result(viz, 'matchingsVizPrev1')
    sys.stderr.write('%d,%d\n'%(sec_ind, sec_ind+1))
    
    if sec1 < last_sec-1:
        sec2 = sec1 + 2
        D_boundaries, D_int, D_ext, D_shape = compute_boundary_distances(sec1, sec2, verbose=False)
        matchings = match_landmarks(sec1, sec2, D=D_boundaries)
        viz = generate_matching_visualizations(sec1, sec2, matchings=matchings)
        dms[sec1].save_pipeline_result(D_boundaries, 'DBoundariesNext2')
        dms[sec1].save_pipeline_result(matchings, 'matchingsNext2')
        dms[sec1].save_pipeline_result(viz, 'matchingsVizNext2')
        dms[sec2].save_pipeline_result(D_boundaries, 'DBoundariesPrev2')
        dms[sec2].save_pipeline_result(matchings, 'matchingsPrev2')
        dms[sec2].save_pipeline_result(viz, 'matchingsVizPrev2')
    sys.stderr.write('%d,%d\n'%(sec_ind, sec_ind+2))
    
    if sec1 < last_sec-2:
        sec2 = sec1 + 3
        D_boundaries, D_int, D_ext, D_shape = compute_boundary_distances(sec1, sec2, verbose=False)
        matchings = match_landmarks(sec1, sec2, D=D_boundaries)
        viz = generate_matching_visualizations(sec1, sec2, matchings=matchings)
        dms[sec1].save_pipeline_result(D_boundaries, 'DBoundariesNext3')
        dms[sec1].save_pipeline_result(matchings, 'matchingsNext3')
        dms[sec1].save_pipeline_result(viz, 'matchingsVizNext3')
        dms[sec2].save_pipeline_result(D_boundaries, 'DBoundariesPrev3')
        dms[sec2].save_pipeline_result(matchings, 'matchingsPrev3')
        dms[sec2].save_pipeline_result(viz, 'matchingsVizPrev3')
    sys.stderr.write('%d,%d\n'%(sec_ind, sec_ind+3))

In [ ]:
Parallel(n_jobs=16)(delayed(f)(sec1) for sec1 in range(first_sec, last_sec+1))

In [ ]:
b = time.time()

for sec1 in range(first_sec, last_sec+1):
    
    if sec1 < last_sec:
        sec2 = sec1 + 1
        D_boundaries, D_int, D_ext, D_shape = compute_boundary_distances(sec1, sec2, verbose=False)
        matchings = match_landmarks(sec1, sec2, D=D_boundaries)
        viz = generate_matching_visualizations(sec1, sec2, matchings=matchings)
        dms[sec1].save_pipeline_result(D_boundaries, 'DBoundariesNext1')
        dms[sec1].save_pipeline_result(matchings, 'matchingsNext1')
        dms[sec1].save_pipeline_result(viz, 'matchingsVizNext1')
        dms[sec2].save_pipeline_result(D_boundaries, 'DBoundariesPrev1')
        dms[sec2].save_pipeline_result(matchings, 'matchingsPrev1')
        dms[sec2].save_pipeline_result(viz, 'matchingsVizPrev1')
    sys.stderr.write('%d,%d\n'%(sec_ind, sec_ind+1))
    
    if sec1 < last_sec-1:
        sec2 = sec1 + 2
        D_boundaries, D_int, D_ext, D_shape = compute_boundary_distances(sec1, sec2, verbose=False)
        matchings = match_landmarks(sec1, sec2, D=D_boundaries)
        viz = generate_matching_visualizations(sec1, sec2, matchings=matchings)
        dms[sec1].save_pipeline_result(D_boundaries, 'DBoundariesNext2')
        dms[sec1].save_pipeline_result(matchings, 'matchingsNext2')
        dms[sec1].save_pipeline_result(viz, 'matchingsVizNext2')
        dms[sec2].save_pipeline_result(D_boundaries, 'DBoundariesPrev2')
        dms[sec2].save_pipeline_result(matchings, 'matchingsPrev2')
        dms[sec2].save_pipeline_result(viz, 'matchingsVizPrev2')
    sys.stderr.write('%d,%d\n'%(sec_ind, sec_ind+2))
    
    if sec1 < last_sec-2:
        sec2 = sec1 + 3
        D_boundaries, D_int, D_ext, D_shape = compute_boundary_distances(sec1, sec2, verbose=False)
        matchings = match_landmarks(sec1, sec2, D=D_boundaries)
        viz = generate_matching_visualizations(sec1, sec2, matchings=matchings)
        dms[sec1].save_pipeline_result(D_boundaries, 'DBoundariesNext3')
        dms[sec1].save_pipeline_result(matchings, 'matchingsNext3')
        dms[sec1].save_pipeline_result(viz, 'matchingsVizNext3')
        dms[sec2].save_pipeline_result(D_boundaries, 'DBoundariesPrev3')
        dms[sec2].save_pipeline_result(matchings, 'matchingsPrev3')
        dms[sec2].save_pipeline_result(viz, 'matchingsVizPrev3')
    sys.stderr.write('%d,%d\n'%(sec_ind, sec_ind+3))
        
print time.time() - b

In [ ]:
must_matches = []

must_match_sections = defaultdict(list)
for m in must_matches:
    mm = sorted(list(m))
    must_match_sections[(mm[0][0], mm[1][0])].append((mm[0][1], mm[1][1]))

In [ ]:
# Match boundaries in consecutive sections

for sec_ind in slide_indices:
    print sec_ind
#     b = time.time()
    if sec_ind < last_sec:
        match_landmarks(sec_ind, sec_ind+1, must_match=must_match_sections[(sec_ind, sec_ind+1)])
    if sec_ind < last_sec-1:
        match_landmarks(sec_ind, sec_ind+2, must_match=must_match_sections[(sec_ind, sec_ind+2)])
    if sec_ind < last_sec-2:
        match_landmarks(sec_ind, sec_ind+3, must_match=must_match_sections[(sec_ind, sec_ind+3)])
#     print time.time()-b